<img style="float: left;" src="earth-lab-logo-rgb.png" width="150" height="150" />

# Homework Template: Earth Analytics Python Course: Spring 2020

Before submitting this assignment, be sure to restart the kernel and run all cells. To do this, pull down the Kernel drop down at the top of this notebook. Then select **restart and run all**.

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below.

* IMPORTANT: Before you submit your notebook, restart the kernel and run all! Your first cell in the notebook should be `[1]` and all cells should run in order! You will lose points if your notebook does not run. 

For all plots and code in general:

* Add appropriate titles to your plot that clearly and concisely describe what the plot shows (e.g. time, location, phenomenon).
* Be sure to use the correct bands for each plot.
* Specify the source of the data for each plot using a plot caption created with `ax.text()`.
* Place ONLY the code needed to create a plot in the plot cells. Place additional processing code ABOVE that cell (in a separate code cell).

Make sure that you:

* **Only include the package imports, code, data, and outputs that are CRUCIAL to your homework assignment.**
* Follow PEP 8 standards. Use the `pep8` tool in Jupyter Notebook to ensure proper formatting (however, note that it does not catch everything!).
* Keep comments concise and strategic. Don't comment every line!
* Organize your code in a way that makes it easy to follow. 
* Write your code so that it can be run on any operating system. This means that:
   1. the data should be downloaded in the notebook to ensure it's reproducible.
   2. all paths should be created dynamically using the os package to ensure that they work across operating systems. 
* Check for spelling errors in your text and code comments


In [1]:
NAME = "Richard Udell"
COLLABORATORS = ""

![Colored Bar](colored-bar.png)

{% include toc title="This Week" icon="file-text" %}

<div class="notice--info" markdown="1">

## <i class="fa fa-ship" aria-hidden="true"></i> Welcome to Week {{ page.week }}!

Welcome to week {{ page.week }} of Earth Analytics! This week you will learn how to automate a workflow using `Python`. You will design and implement your own workflow in `Python` that builds on the skills that you have learned in this course, such as functions and loops. You will also learn how to programmatically build paths to directories and files as well as parse strings to extract information from file and directory names.  

{% include/data_subsets/course_earth_analytics/_data-landsat-automation.md %}


</div>


## Automate a Workflow in Python

For this week’s assignment, you will generate a plot of the normalized difference vegetation index (NDVI) for two different locations in the United States to begin to understand how the growing seasons vary in each site:

1. <a href="https://www.neonscience.org/field-sites/field-sites-map/SJER" target="_blank">San Joaquin Experimental Range (SJER) in Southern California, United States</a>
2. <a href="https://www.neonscience.org/field-sites/field-sites-map/HARV" target="_blank">Harvard Forest (HARV) in the Eastern United States</a> 

From this plot, you will be able to compare the seasonal vegetation patterns of the two locations. This comparison would be useful if you were planning NEON’s upcoming flight season in both locations and wanted to ensure that you flew the area when the vegetation was the most green! If could also be useful if you wanted to track green-up as it happened over time in both sites to see if there were changes happening. 

As a bonus, you will also create a stacked NDVI output data product to share with your colleagues. You are doing all of the work to clean and process the data. It would be nice if you could share a data product output to save others the hassle. 

## Design A Workflow 

Your goal this week is to calculate the mean NDVI value for each Landsat 8 scene captured for a NEON site over a year. You have the following data to do accomplish this goal:

1. One year worth of Landsat 8 data for each site: Remember that for each landsat scene, you have a series of geotiff files representing bands and qa (quality assurance) layers in your data.
2. A site boundary “clip file” for each site: This is a shapefile representing the boundary of each NEON site. You will want to clip your landsat data to this boundary.

Before writing `Python` code, write pseudocode for your implementation. Pseudo-coding means that you will write out all of the steps that you need to perform. Then, you will identify areas where tasks are repeated that could benefit from a function, areas where loops might be appropriate, etc.  


## Homework for this Week

Your homework for this week is provided in the assignment dropbox on Canvas.


## Homework Plots

The plots below are examples of what your output plots will look like with and without dealing with clouds.


In [2]:
# Autograding imports - do not modify this cell

import matplotcheck.autograde as ag
import matplotcheck.notebook as nb
import matplotcheck.timeseries as ts
from datetime import datetime

In [3]:
# Import necessary packages
import os
from glob import glob
import matplotlib.pyplot as plt
from matplotlib import patches as mpatches, colors
import numpy as np
from shapely.geometry import box
import geopandas as gpd
import rasterio as rio
from rasterio.plot import plotting_extent
from rasterio.mask import mask
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import earthpy.mask as em
import pandas as pd
import geopandas as gpd

# Imports for box function?


# Get data and set working directory
data = et.data.get_data('ndvi-automation')
os.chdir(os.path.join(et.io.HOME, 'earth-analytics'))

In [4]:
# Import SJER site boundary (.shp)
sjer_base_path = os.path.join('data', 'ndvi-automation', 'sites', 'SJER')
sjer_site_boundary_path = os.path.join(
    sjer_base_path, 'vector', 'SJER-crop.shp')
sjer_site_boundary = gpd.read_file(sjer_site_boundary_path, masked=True)

# Import HARV site boundary (.shp)
harv_base_path = os.path.join('data', 'ndvi-automation', 'sites', 'HARV')
harv_site_boundary_path = os.path.join(
    harv_base_path, 'vector', 'HARV-crop.shp')
harv_site_boundary = gpd.read_file(harv_site_boundary_path, masked=True)

### Best attempt at automation

In [5]:
# # Make a list of directories via glob
# path = os.path.join('data','ndvi-automation','*','*','landsat-crop','*')
# all_dirs = glob(path)

In [6]:
# # Automation baby!!!
# all_ndvi = []

# for directory in all_dirs:
#     # Extract the date of the directory 
#     date = os.path.basename(os.path.normpath(directory))
#     date = date[10:18]
#     all_ndvi.append(date)
    
#     # Extract qa and open via context manager
#     qa_glob = glob(os.path.join(directory + '/*pixel_qa*.tif'))
#     with rio.open(qa_glob[0]) as qa_src:
#         qa = qa_src.read(1)
    
#     # Exctract all_bands(sorted) via glob from each directory
#     all_bands_glob = glob(os.path.join(directory + '/*band*.tif'))
#     all_bands_glob.sort()
    
#     # Run stack function
#     output_path = os.path.join('data','ndvi-automation','outputs','file.tif')
#     land_stack, land_meta = es.stack(all_bands_glob, output_path)
    
     
#     # Open stack ouput .tif and read all_bands via context manager
#     with rio.open(output_path) as bands_src:
#         all_bands = bands_src.read() # Missing cloud management (come back to this)
#                                # Also missing: clipping to the site boundary (come back to this)
    
# #     # Mask for cloud cover
# #     high_cloud_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cloud Confidence"]
# #     all_bands_free = em.mask_pixels(arr=all_bands, mask_arr=qa, vals=high_cloud_confidence)
    
    
#     # Calculate NDVI
#     ndvi = es.normalized_diff(all_bands[4], all_bands[3]) 
    
#     # Save ndvi's to this list
#     all_ndvi.append(ndvi)
    
    
# len(all_ndvi) 

### Work with one scene for one site

In [7]:
# Make a list of directories via glob
path = os.path.join('data','ndvi-automation','*','SJER','landsat-crop','*')
SJER_dirs = glob(path)

path = os.path.join('data','ndvi-automation','*','HARV','landsat-crop','*')
HARV_dirs = glob(path)

In [8]:
# Here is the masking variable we will use
# Create a list of values that you want to set as "mask" in the pixel qa layer   
high_cloud_confidence = em.pixel_flags["pixel_qa"]["L8"]["High Cloud Confidence"]
cloud = em.pixel_flags["pixel_qa"]["L8"]["Cloud"]
cloud_shadow = em.pixel_flags["pixel_qa"]["L8"]["Cloud Shadow"]

all_masked_values = cloud_shadow + cloud + high_cloud_confidence

In [9]:
HARV_dirs[0]

'data/ndvi-automation/sites/HARV/landsat-crop/LC080130302017041801T1-SC20181023152618'

In [10]:
# Here is the dictionary that we will populate with the calc_ndvi_mean
#fucntion
dict_to_df = {}


def calc_ndvi_mean(list_of_dirs, site_boundary_shp):  # Still needs docstring...help-text
    """Loop through given list of Landsat8 directories to clip each to the given site
    boundary, extract the date and site name, and calculate the mean NDVI for each scene.

    Parameters
    ----------
    list_of_dirs : list
        List of Landsat8 scene directories in Landsat Collection 1
        Level-1 product identifier format: 
        LXSS_LLLL_PPPRRR_YYYYMMDD_yyyymmdd_CC_TX.

    site_boundary_shp : geopandas.GeoDataFrame
        .shp file imported into a GeoDataFrame.  Site boundary
        must be entirely within the Landsat8 scene.

    Returns
    ------
    dict_to_df : dictionary
        Dictionary with the scene date as the key and the mean NDVI
        and the site name as the value pairs.
    """

    for directory in list_of_dirs:

        # Extract the site name
        site_name = directory[27:31]

        # Extract the date of the directory
        date = os.path.basename(os.path.normpath(directory))
        date = date[10:18]

        # Extract qa and open via context manager
        qa_glob = glob(os.path.join(directory + '/*pixel_qa*.tif'))
        with rio.open(qa_glob[0]) as qa_src:
            qa, qa_meta = es.crop_image(qa_src, site_boundary_shp)
            #qa = qa_src.read(1)
            qa_extent = plotting_extent(qa_src)  # Is this code necessary?

        # Exctract all_bands(sorted) via glob from the directory
        all_bands_glob = glob(os.path.join(directory + '/*band*.tif'))
        all_bands_glob.sort()

        # Run stack function
        output_path = os.path.join(
            'data', 'ndvi-automation', 'outputs', 'all_bands.tif')
        land_stack, land_meta = es.stack(all_bands_glob, output_path)

        # Open stack ouput .tif and read all_bands via context manager
        with rio.open(output_path) as bands_src:
            all_bands, all_bands_meta = es.crop_image(
                bands_src, site_boundary_shp)

        # Prepare masked values variable to only include mask values in qa layer
        active_cloud_codes = []

        for cloud_code in all_masked_values:
            if np.any(cloud_code == qa):
                active_cloud_codes.append(cloud_code)

        # Eliminates duplicates
        active_cloud_codes = list(set(active_cloud_codes))

        # Masking
        # Mask all_bands with qa layer if necessary and calculate NDVI
        if len(active_cloud_codes) != 0:
            all_bands_masked = em.mask_pixels(arr=all_bands,
                                              mask_arr=qa,
                                              vals=active_cloud_codes)

            ndvi = es.normalized_diff(all_bands_masked[4], all_bands_masked[3])
        else:
            ndvi = ndvi = es.normalized_diff(all_bands[4], all_bands[3])

        mean_ndvi = ndvi.mean()

        # Now let's get that dictionary going
        # Add date as key and add value pair as ndvi.mean() and site_name
        dict_to_df.update({date: (mean_ndvi, site_name)})
        
    return dict_to_df


In [11]:
calc_ndvi_mean(SJER_dirs, sjer_site_boundary)
calc_ndvi_mean(HARV_dirs, harv_site_boundary)

{'20170904': (masked, 'SJER'),
 '20170819': (0.32750823563090536, 'SJER'),
 '20171022': (0.31722907031043807, 'SJER'),
 '20171107': (0.3137839639712589, 'SJER'),
 '20170702': (0.3346638151749849, 'SJER'),
 '20170107': (masked, 'SJER'),
 '20170920': (0.33103548523629656, 'SJER'),
 '20170515': (0.441691751593773, 'SJER'),
 '20170429': (0.6104344457442622, 'SJER'),
 '20171123': (0.32494695680446906, 'SJER'),
 '20170616': (0.3587106397670503, 'SJER'),
 '20170312': (0.663938167189907, 'SJER'),
 '20170208': (masked, 'SJER'),
 '20170803': (masked, 'SJER'),
 '20170123': (masked, 'SJER'),
 '20171209': (0.35229998591810596, 'SJER'),
 '20171225': (0.27246451352603895, 'SJER'),
 '20170531': (masked, 'SJER'),
 '20171006': (0.3055060260766697, 'SJER'),
 '20170224': (0.663272893252757, 'SJER'),
 '20170413': (masked, 'SJER'),
 '20170328': (0.7029135204276337, 'SJER'),
 '20170718': (0.31988653659400057, 'SJER'),
 '20170418': (0.5418116737594111, 'HARV'),
 '20170317': (0.28402842011750823, 'HARV'),
 '20

In [12]:
mean_ndvi_2017_df = pd.DataFrame.from_dict(
    dict_to_df, orient='index', columns=['mean_NDVI', 'site_name'])

In [18]:
# Let's take a look at the dataframe
mean_ndvi_2017_df

,mean_NDVI,site_name
20170904,--,SJER
20170819,0.327508,SJER
20171022,0.317229,SJER
20171107,0.313784,SJER
20170702,0.334664,SJER
20170107,--,SJER
20170920,0.331035,SJER
20170515,0.441692,SJER
20170429,0.610434,SJER
20171123,0.324947,SJER


In [13]:
# Next step is to clean up and plot the dataframe!

In [ ]:
ep.plot_bands(unmasked_ndvi, cmap="Reds", scale=False)

In [ ]:
# Test your dataframe! For each test, rename the dataframe comparison to check against your dataframe.

# For example, in test one, the date test, change "student_dataframe['date_column']" to match
# your dataframe's name and date column. If your dataframe was called ndvi_data and the dates column was named
# date_time, than the lines should look like `assert date_series.equals(ndvi_data['date_time'].sort_values())`

# These tests are not graded so if they fail but your graph is correct
# you won't be docked any points! Leave the sort values statement, only replace the dataframe and column names.

# This is for data that hasn't been masked yet, it should be a half way sanity check.

try:
    assert site_series.equals(ndvi_ts['site'].sort_values())
    print('Your site names appear to be accurate!')
except AssertionError:
    print('There seems to be an issue with your site names, check that column to make sure the values are correct.')
try:
    assert date_series.equals(ndvi_ts['date'].sort_values())
    print('Your date values appear to be accurate!')
except AssertionError:
    print('There seems to be an issue with your date values, check that column to make sure the values are correct.')
try:
    assert ndvi_series.equals(ndvi_ts['ndvi'].sort_values())
    print('Your ndvi values appear to be accurate!')
except AssertionError:
    print('There seems to be an issue with your ndvi values, check that column to make sure the values are correct.')

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### DO NOT REMOVE LINE BELOW ###
final_masked_solution = nb.convert_axes(plt, which_axes="current")